In [108]:
%load_ext autoreload
%autoreload 2
!export PYTHONPATH=$PYTHONPATH:/Users/arvindagarwal/Desktop/cmu/courses/ml/lda2vec-pytorch/utils


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.insert(0,'.')
sys.path.insert(0,'/Users/arvindagarwal/Desktop/cmu/courses/ml/lda2vec-pytorch/utils')
# from utils import preprocess, get_windows
# from utils.preprocess import preprocess
from preprocess_mod import *
from get_windows_mod import *

In [110]:
!ls ..

20newsgroups LICENSE      README.md    loss.png     utils


In [100]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [101]:
nlp = spacy.load('en')

# Load dataset

In [102]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [103]:
# number of documents
len(docs)

18846

In [104]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [105]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)




  0%|          | 0/18846 [00:00<?, ?it/s]


  0%|          | 1/18846 [00:00<1:19:40,  3.94it/s]


  0%|          | 3/18846 [00:00<1:01:53,  5.07it/s]


  0%|          | 6/18846 [00:00<47:03,  6.67it/s]  


  0%|          | 9/18846 [00:00<36:06,  8.70it/s]


  0%|          | 13/18846 [00:00<34:10,  9.18it/s]


  0%|          | 17/18846 [00:01<27:02, 11.61it/s]


  0%|          | 19/18846 [00:01<31:56,  9.82it/s]


  0%|          | 21/18846 [00:01<32:51,  9.55it/s]


  0%|          | 23/18846 [00:01<30:17, 10.36it/s]


  0%|          | 25/18846 [00:01<28:07, 11.15it/s]


  0%|          | 28/18846 [00:02<23:02, 13.61it/s]


  0%|          | 30/18846 [00:02<24:57, 12.56it/s]


  0%|          | 34/18846 [00:02<20:02, 15.64it/s]


  0%|          | 39/18846 [00:02<16:56, 18.49it/s]


  0%|          | 42/18846 [00:02<15:01, 20.85it/s]


  0%|          | 45/18846 [00:03<30:30, 10.27it/s]


  0%|          | 48/18846 [00:03<24:37, 12.73it/s]


  0%|          | 51/18846 [00:03<21:17, 14.71it/s]

  2%|▏         | 442/18846 [00:22<20:17, 15.11it/s]


  2%|▏         | 445/18846 [00:22<18:36, 16.48it/s]


  2%|▏         | 448/18846 [00:23<19:44, 15.53it/s]


  2%|▏         | 450/18846 [00:23<21:22, 14.34it/s]


  2%|▏         | 452/18846 [00:23<25:06, 12.21it/s]


  2%|▏         | 454/18846 [00:23<23:02, 13.30it/s]


  2%|▏         | 457/18846 [00:23<19:12, 15.96it/s]


  2%|▏         | 459/18846 [00:23<18:28, 16.58it/s]


  2%|▏         | 464/18846 [00:24<15:08, 20.24it/s]


  2%|▏         | 467/18846 [00:24<14:49, 20.66it/s]


  2%|▏         | 470/18846 [00:24<14:44, 20.77it/s]


  3%|▎         | 474/18846 [00:24<12:42, 24.10it/s]


  3%|▎         | 480/18846 [00:24<10:39, 28.70it/s]


  3%|▎         | 485/18846 [00:24<09:31, 32.14it/s]


  3%|▎         | 489/18846 [00:24<09:20, 32.77it/s]


  3%|▎         | 493/18846 [00:24<10:47, 28.35it/s]


  3%|▎         | 497/18846 [00:25<12:16, 24.91it/s]


  3%|▎         | 500/18846 [00:25<15:00, 20.37it/s]


  3%|▎         | 504/18846 [

  5%|▍         | 900/18846 [00:48<13:37, 21.95it/s]


  5%|▍         | 905/18846 [00:48<14:09, 21.11it/s]


  5%|▍         | 910/18846 [00:48<12:15, 24.38it/s]


  5%|▍         | 913/18846 [00:48<12:28, 23.96it/s]


  5%|▍         | 916/18846 [00:48<13:11, 22.66it/s]


  5%|▍         | 920/18846 [00:48<11:43, 25.49it/s]


  5%|▍         | 924/18846 [00:48<10:43, 27.85it/s]


  5%|▍         | 929/18846 [00:49<10:49, 27.57it/s]


  5%|▍         | 934/18846 [00:49<09:26, 31.63it/s]


  5%|▍         | 938/18846 [00:49<11:13, 26.57it/s]


  5%|▌         | 943/18846 [00:49<09:47, 30.45it/s]


  5%|▌         | 947/18846 [00:49<09:14, 32.29it/s]


  5%|▌         | 952/18846 [00:49<08:36, 34.62it/s]


  5%|▌         | 956/18846 [00:49<08:21, 35.69it/s]


  5%|▌         | 960/18846 [00:50<11:31, 25.88it/s]


  5%|▌         | 964/18846 [00:50<10:42, 27.84it/s]


  5%|▌         | 968/18846 [00:50<14:00, 21.27it/s]


  5%|▌         | 971/18846 [00:50<13:18, 22.39it/s]


  5%|▌         | 975/18846 [

  7%|▋         | 1364/18846 [01:12<33:15,  8.76it/s]


  7%|▋         | 1368/18846 [01:12<26:04, 11.17it/s]


  7%|▋         | 1372/18846 [01:12<21:42, 13.42it/s]


  7%|▋         | 1375/18846 [01:12<18:39, 15.61it/s]


  7%|▋         | 1378/18846 [01:12<19:19, 15.06it/s]


  7%|▋         | 1382/18846 [01:12<16:05, 18.09it/s]


  7%|▋         | 1385/18846 [01:13<17:56, 16.23it/s]


  7%|▋         | 1388/18846 [01:13<18:04, 16.10it/s]


  7%|▋         | 1391/18846 [01:13<16:31, 17.61it/s]


  7%|▋         | 1394/18846 [01:13<25:28, 11.42it/s]


  7%|▋         | 1396/18846 [01:13<23:55, 12.15it/s]


  7%|▋         | 1398/18846 [01:14<22:41, 12.82it/s]


  7%|▋         | 1400/18846 [01:14<27:32, 10.56it/s]


  7%|▋         | 1403/18846 [01:14<22:45, 12.77it/s]


  7%|▋         | 1407/18846 [01:14<18:38, 15.60it/s]


  7%|▋         | 1410/18846 [01:14<18:43, 15.52it/s]


  7%|▋         | 1413/18846 [01:14<16:15, 17.87it/s]


  8%|▊         | 1417/18846 [01:15<13:54, 20.88it/s]


  8%|▊    

 10%|▉         | 1818/18846 [01:33<19:37, 14.46it/s]


 10%|▉         | 1821/18846 [01:34<21:57, 12.92it/s]


 10%|▉         | 1824/18846 [01:34<18:25, 15.40it/s]


 10%|▉         | 1828/18846 [01:34<15:19, 18.51it/s]


 10%|▉         | 1831/18846 [01:34<14:31, 19.52it/s]


 10%|▉         | 1834/18846 [01:34<13:49, 20.52it/s]


 10%|▉         | 1837/18846 [01:34<13:18, 21.31it/s]


 10%|▉         | 1842/18846 [01:34<11:10, 25.36it/s]


 10%|▉         | 1846/18846 [01:35<10:22, 27.31it/s]


 10%|▉         | 1850/18846 [01:35<09:40, 29.26it/s]


 10%|▉         | 1854/18846 [01:35<09:06, 31.11it/s]


 10%|▉         | 1858/18846 [01:35<11:27, 24.72it/s]


 10%|▉         | 1861/18846 [01:35<20:41, 13.68it/s]


 10%|▉         | 1864/18846 [01:36<19:29, 14.52it/s]


 10%|▉         | 1867/18846 [01:36<19:26, 14.55it/s]


 10%|▉         | 1871/18846 [01:36<16:52, 16.77it/s]


 10%|▉         | 1874/18846 [01:36<14:52, 19.02it/s]


 10%|▉         | 1877/18846 [01:36<14:23, 19.65it/s]


 10%|▉    

...

 91%|█████████ | 17068/18846 [15:27<00:57, 31.17it/s]


 91%|█████████ | 17072/18846 [15:27<00:59, 30.04it/s]


 91%|█████████ | 17076/18846 [15:27<01:03, 27.66it/s]


 91%|█████████ | 17082/18846 [15:27<00:54, 32.27it/s]


 91%|█████████ | 17086/18846 [15:27<01:03, 27.58it/s]


 91%|█████████ | 17091/18846 [15:28<00:55, 31.69it/s]


 91%|█████████ | 17095/18846 [15:28<00:54, 31.99it/s]


 91%|█████████ | 17099/18846 [15:28<01:01, 28.54it/s]


 91%|█████████ | 17104/18846 [15:28<00:54, 31.86it/s]


 91%|█████████ | 17108/18846 [15:28<00:52, 33.11it/s]


 91%|█████████ | 17112/18846 [15:28<00:50, 34.17it/s]


 91%|█████████ | 17116/18846 [15:28<00:52, 32.75it/s]


 91%|█████████ | 17121/18846 [15:28<00:47, 36.34it/s]


 91%|█████████ | 17125/18846 [15:29<00:57, 30.00it/s]


 91%|█████████ | 17130/18846 [15:29<00:50, 34.02it/s]


 91%|█████████ | 17134/18846 [15:29<00:54, 31.32it/s]


 91%|█████████ | 17138/18846 [15:29<01:47, 15.89it/s]


 91%|█████████ | 17142/18846 [15:30<01:30, 18.74

 93%|█████████▎| 17549/18846 [15:47<01:20, 16.18it/s]


 93%|█████████▎| 17553/18846 [15:47<01:27, 14.69it/s]


 93%|█████████▎| 17556/18846 [15:47<01:14, 17.31it/s]


 93%|█████████▎| 17559/18846 [15:48<01:09, 18.54it/s]


 93%|█████████▎| 17562/18846 [15:48<01:02, 20.69it/s]


 93%|█████████▎| 17569/18846 [15:48<00:50, 25.26it/s]


 93%|█████████▎| 17573/18846 [15:48<00:49, 25.89it/s]


 93%|█████████▎| 17577/18846 [15:48<00:45, 28.09it/s]


 93%|█████████▎| 17581/18846 [15:48<00:42, 29.92it/s]


 93%|█████████▎| 17585/18846 [15:49<01:14, 16.91it/s]


 93%|█████████▎| 17589/18846 [15:49<01:03, 19.94it/s]


 93%|█████████▎| 17592/18846 [15:49<01:01, 20.35it/s]


 93%|█████████▎| 17595/18846 [15:49<01:15, 16.67it/s]


 93%|█████████▎| 17598/18846 [15:49<01:16, 16.34it/s]


 93%|█████████▎| 17604/18846 [15:50<01:01, 20.32it/s]


 93%|█████████▎| 17607/18846 [15:50<01:02, 19.92it/s]


 93%|█████████▎| 17612/18846 [15:50<00:51, 23.95it/s]


 93%|█████████▎| 17616/18846 [15:50<01:04, 19.17

 96%|█████████▌| 18021/18846 [16:09<01:28,  9.34it/s]


 96%|█████████▌| 18024/18846 [16:09<01:51,  7.37it/s]


 96%|█████████▌| 18028/18846 [16:09<01:27,  9.32it/s]


 96%|█████████▌| 18033/18846 [16:09<01:07, 12.08it/s]


 96%|█████████▌| 18036/18846 [16:10<01:01, 13.23it/s]


 96%|█████████▌| 18039/18846 [16:10<00:53, 15.03it/s]


 96%|█████████▌| 18042/18846 [16:10<00:58, 13.84it/s]


 96%|█████████▌| 18045/18846 [16:10<00:48, 16.47it/s]


 96%|█████████▌| 18048/18846 [16:10<00:51, 15.45it/s]


 96%|█████████▌| 18051/18846 [16:11<00:46, 17.14it/s]


 96%|█████████▌| 18054/18846 [16:11<00:42, 18.48it/s]


 96%|█████████▌| 18057/18846 [16:11<00:38, 20.31it/s]


 96%|█████████▌| 18060/18846 [16:11<00:39, 19.71it/s]


 96%|█████████▌| 18063/18846 [16:11<00:44, 17.51it/s]


 96%|█████████▌| 18067/18846 [16:11<00:37, 20.73it/s]


 96%|█████████▌| 18070/18846 [16:11<00:36, 20.98it/s]


 96%|█████████▌| 18074/18846 [16:12<00:43, 17.76it/s]


 96%|█████████▌| 18079/18846 [16:12<00:35, 21.56

 98%|█████████▊| 18516/18846 [16:36<00:13, 24.75it/s]


 98%|█████████▊| 18520/18846 [16:36<00:11, 27.45it/s]


 98%|█████████▊| 18527/18846 [16:36<00:09, 32.59it/s]


 98%|█████████▊| 18532/18846 [16:36<00:09, 34.82it/s]


 98%|█████████▊| 18536/18846 [16:37<00:10, 28.25it/s]


 98%|█████████▊| 18540/18846 [16:37<00:10, 30.01it/s]


 98%|█████████▊| 18544/18846 [16:37<00:12, 23.36it/s]


 98%|█████████▊| 18547/18846 [16:37<00:12, 24.19it/s]


 98%|█████████▊| 18550/18846 [16:37<00:12, 24.51it/s]


 98%|█████████▊| 18554/18846 [16:37<00:10, 26.58it/s]


 98%|█████████▊| 18557/18846 [16:37<00:10, 26.63it/s]


 98%|█████████▊| 18563/18846 [16:38<00:09, 30.82it/s]


 99%|█████████▊| 18567/18846 [16:38<00:09, 28.79it/s]


 99%|█████████▊| 18572/18846 [16:38<00:08, 31.34it/s]


 99%|█████████▊| 18576/18846 [16:38<00:09, 27.78it/s]


 99%|█████████▊| 18580/18846 [16:38<00:08, 29.80it/s]


 99%|█████████▊| 18584/18846 [16:38<00:11, 23.78it/s]


 99%|█████████▊| 18587/18846 [16:39<00:14, 18.27

number of removed short documents: 3979
total number of tokens: 1441495
number of tokens to be removed: 414060
number of additionally removed short documents: 2189
total number of tokens: 1002444

minimum word count number: 14
this number can be less than MIN_COUNTS because of document removal


In [111]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [112]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')




0it [00:00, ?it/s]


501it [00:00, 4984.62it/s]


901it [00:00, 2373.41it/s]


1534it [00:00, 2921.11it/s]


1870it [00:00, 2582.74it/s]


2460it [00:00, 3088.34it/s]


2814it [00:01, 2683.71it/s]


3288it [00:01, 3070.86it/s]


3644it [00:01, 2401.66it/s]


4077it [00:01, 2771.70it/s]


4640it [00:01, 2514.85it/s]


5380it [00:01, 3132.98it/s]


5986it [00:01, 3663.66it/s]


6469it [00:02, 2755.83it/s]


7079it [00:02, 3297.89it/s]


7668it [00:02, 3799.10it/s]


8164it [00:02, 2442.53it/s]


8869it [00:02, 3038.14it/s]


9616it [00:02, 3695.38it/s]


10225it [00:03, 4189.31it/s]


10801it [00:03, 2758.77it/s]


11333it [00:03, 3190.36it/s]


11924it [00:03, 3701.23it/s]


12607it [00:03, 4290.33it/s]


12678it [00:03, 3372.08it/s]

In [113]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [114]:
# number of windows (equals to the total number of tokens)
data.shape[0]

1002444

# Get unigram distribution

In [128]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [129]:
# %%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

In [130]:
# number of unique words
vocab_size

7854

# Prepare initialization for document weights

In [131]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [132]:
# %%time
n_topics = 25
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

In [133]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : section error echo return build GIZ function output rule character
topic 1 : vote neutral ground group discussion pay sure offer actually fielder
topic 2 : posting newsgroup list group request internet mail address email message
topic 3 : drug increase patient report cause age study disease child research
topic 4 : window application widget Motif server Sun mouse client manager screen
topic 5 : President job country state talk States force United today decision
topic 6 : RIPEM space datum NASA public launch DES Space satellite RSA
topic 7 : fire child FBI happen hear kill Koresh leave door house
topic 8 : bike light ride hole turn mile road little far black
topic 9 : Jesus Lord Christ man Bible word shall Psalms sin John
topic 10 : law encryption chip technology device agency escrow phone public privacy
topic 11 : play team period goal Vancouver shot Detroit series NHL win
topic 12 : dog restraint sure sperm lot hear Register Weaver Khomeini Rushdie
topic 13 : card disk drive

In [134]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob




  0%|          | 0/12678 [00:00<?, ?it/s]


  1%|          | 99/12678 [00:00<00:12, 989.74it/s]


  1%|▏         | 172/12678 [00:00<00:13, 893.85it/s]


  2%|▏         | 276/12678 [00:00<00:13, 930.75it/s]


  3%|▎         | 410/12678 [00:00<00:11, 1023.10it/s]


  4%|▍         | 535/12678 [00:00<00:11, 1080.83it/s]


  5%|▌         | 660/12678 [00:00<00:10, 1125.03it/s]


  6%|▌         | 764/12678 [00:00<00:11, 1014.70it/s]


  7%|▋         | 865/12678 [00:00<00:11, 1013.20it/s]


  8%|▊         | 995/12678 [00:00<00:10, 1084.67it/s]


  9%|▊         | 1103/12678 [00:01<00:10, 1075.22it/s]


 10%|▉         | 1241/12678 [00:01<00:09, 1150.48it/s]


 11%|█         | 1369/12678 [00:01<00:09, 1185.14it/s]


 12%|█▏        | 1502/12678 [00:01<00:09, 1224.29it/s]


 13%|█▎        | 1626/12678 [00:01<00:09, 1170.15it/s]


 14%|█▍        | 1771/12678 [00:01<00:08, 1241.16it/s]


 15%|█▍        | 1898/12678 [00:01<00:09, 1162.32it/s]


 16%|█▌        | 2017/12678 [00:01<00:10, 970.85it/s] 

# Save data

In [135]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)